In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [4]:
imgPaths = glob("data/selected/*.jpeg")

imgPaths[:5]

['data/selected\\000.jpeg',
 'data/selected\\001.jpeg',
 'data/selected\\002.jpeg',
 'data/selected\\003.jpeg',
 'data/selected\\004.jpeg']

In [8]:
imgpath = imgPaths[0]
print(imgpath)

_, filename = os.path.split(imgpath)
print(filename)

data/selected\000.jpeg
000.jpeg


In [11]:
imgpath = imgPaths[0]
#print(imgpath)

_, filename = os.path.split(imgpath)
#print(filename)

# extract data and text
image = cv2.imread(imgpath)
data = pytesseract.image_to_data(image)

dataList = list(map(lambda x: x.split("\t"), data.split("\n")))
df = pd.DataFrame(dataList[1:], columns=dataList[0])
df.dropna(inplace=True)

In [12]:
df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,1800,1200,-1,
1,2,1,1,0,0,0,0,0,1800,1,-1,
2,3,1,1,1,0,0,0,0,1800,1,-1,
3,4,1,1,1,1,0,0,0,1800,1,-1,
4,5,1,1,1,1,1,0,0,1800,1,95,


In [14]:
imgpath = imgPaths[0]
#print(imgpath)

_, filename = os.path.split(imgpath)
#print(filename)

# extract data and text
image = cv2.imread(imgpath)
data = pytesseract.image_to_data(image)

dataList = list(map(lambda x: x.split("\t"), data.split("\n")))
df = pd.DataFrame(dataList[1:], columns=dataList[0])
df.dropna(inplace=True)
df["conf"] = df["conf"].astype(int)

usefulData = df.query("conf >= 30")

#DataFrame which contains the usefull information that is required for labeling
businessCard = pd.DataFrame()
businessCard["text"] = usefulData["text"]
businessCard["id"] = filename

In [15]:
businessCard

,text,id
4,,000.jpeg
8,.,000.jpeg
9,040-4852,000.jpeg
10,"8881,",000.jpeg
11,90309,000.jpeg
12,52549,000.jpeg
14,Fi,000.jpeg
15,/laurelsoverseaseducation,000.jpeg
20,LAURELS,000.jpeg
24,OVERSEAS,000.jpeg


In [16]:
allBusinessCard = pd.DataFrame(columns=["id", "text"])

for imgpath in tqdm(imgPaths, desc="BusinessCard"):
    
    #imgpath = imgPaths[0]
    #print(imgpath)

    _, filename = os.path.split(imgpath)
    #print(filename)

    # extract data and text
    image = cv2.imread(imgpath)
    data = pytesseract.image_to_data(image)

    dataList = list(map(lambda x: x.split("\t"), data.split("\n")))
    df = pd.DataFrame(dataList[1:], columns=dataList[0])
    df.dropna(inplace=True)
    df["conf"] = df["conf"].astype(int)

    usefulData = df.query("conf >= 30")

    #DataFrame which contains the usefull information that is required for labeling
    businessCard = pd.DataFrame()
    businessCard["text"] = usefulData["text"]
    businessCard["id"] = filename
    
    # Concatenation
    allBusinessCard = pd.concat((allBusinessCard, businessCard))

BusinessCard: 100%|██████████████████████████████████████████████████████████████████| 293/293 [10:30<00:00,  2.15s/it]


In [18]:
allBusinessCard.to_csv("businessCard.csv", index=False)